In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot as plt

import os,gc
from tqdm.auto import tqdm
import pickle
import re

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

import missingno as msno
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests
from statsmodels.tools.sm_exceptions import PerfectSeparationError

In [ ]:
%%time
path="/kaggle/input/hull-tactical-market-prediction"
samples=[]
file_path=f"{path}/train.csv"
part=pd.read_csv(file_path)
samples.append(part)
sample_df=pd.concat(samples,ignore_index=True)
sample_df.round(1)

In [ ]:
sample_df.columns

In [ ]:
sample_df.isnull().sum()

In [ ]:
sample_df["forward_returns"].mean()

In [ ]:
rolling_var = sample_df["forward_returns"].rolling(window=5).var()
plt.figure(figsize=(15,8))
plt.plot(rolling_var, label=f'Rolling Variance forward returns',color="black")
plt.title(f'Rolling Variance of forward_returns')
plt.xlabel('Time')
plt.ylabel('Variance')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

sample_df['date_id'] = pd.to_datetime(sample_df['date_id'])
sample_df = sample_df.set_index('date_id', drop=False)

plt.figure(figsize=(12,6))
plt.plot(sample_df.index, sample_df['forward_returns'], label='Forward Returns',color="black")
plt.xlabel("Date")
plt.ylabel("Forward Returns")
plt.title("Time Series Plot")
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(sample_df.index,sample_df["risk_free_rate"],label="Risk Free Rate",color="black")
plt.xlabel("Date")
plt.ylabel("Risk Free Rate")
plt.title("Time series plot")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(sample_df.index,sample_df["market_forward_excess_returns"],label="market_forward_excess_returns",color="black")
plt.xlabel("Date")
plt.ylabel("Market Forward access returns")
plt.title("Time series plot")
plt.legend()
plt.show()

In [ ]:
pd.infer_freq(sample_df["date_id"])

In [ ]:
plt.hist(sample_df["forward_returns"],bins=30,color="black",edgecolor="lightgrey")
plt.title("Forward Returns Distribution")
plt.xlabel("Forward Returns")
plt.ylabel("frequency")
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(sample_df["date_id"],sample_df["forward_returns"],color="black",s=5)
plt.title("Time Series Scatter Plot")
plt.xlabel("Date")
plt.ylabel("Forward Returns")
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(sample_df["date_id"],sample_df["risk_free_rate"],color="black",s=5)
plt.title("Risk Free Rate")
plt.xlabel("Date")
plt.ylabel("Risk Free Rate")
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(sample_df["date_id"],sample_df["market_forward_excess_returns"],color="black",s=5)
plt.title("market_forward_excess_returns")
plt.xlabel("Date")
plt.ylabel("market_forward_excess_returns")
plt.show()

In [ ]:
sample_df["diff"]=sample_df["forward_returns"].diff()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(sample_df['date_id'], sample_df['diff'], color='black', s=5)
plt.title("Scatter Plot of Differences in Time Series")
plt.xlabel("Date")
plt.ylabel("Difference in Value")
plt.axhline(0, color='lightgrey', linestyle='--') 
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(sample_df['forward_returns'].shift(1), sample_df['diff'],color="black", s=10)
plt.xlabel("Previous Value")
plt.ylabel("Difference")
plt.title("Difference vs Previous Value")
plt.show()

In [ ]:
sample_df['date_id'] = sample_df['date_id'].astype('int64') // 10**9

In [ ]:
na_counts = sample_df.isnull().sum()
na_counts = na_counts[na_counts > 0].sort_values(ascending=False)
plt.figure(figsize=(24,9))
na_counts.plot(kind="bar",color="darkred")
plt.title("Missing Values per Feature")
plt.ylabel("Count")
plt.show()


In [ ]:
import seaborn as sns
plt.figure(figsize=(18,9))
sns.heatmap(sample_df.isnull(),cbar=False,cmap="viridis")
plt.show()

In [ ]:
missing_cols = sample_df.columns[sample_df.isnull().any()]
msno.bar(sample_df[missing_cols])


In [ ]:
missing_percent = sample_df.isnull().mean().sort_values(ascending=False) * 100
plt.figure(figsize=(12,6))
sns.barplot(x=missing_percent.index[:20], y=missing_percent.values[:20])
plt.xticks(rotation=90)
plt.ylabel("Percentage Missing")
plt.show()


# VISUALIZING SUCH A BIG DATASET DOES NOT GIVE MUCH INFORMATION!lets perform LITTLE'S MCAR Test


In [ ]:
sample_df.dtypes

In [ ]:
def cohens_d(a, b):
    a, b = np.array(a), np.array(b)
    n1, n2 = len(a), len(b)
    s1, s2 = a.std(ddof=1), b.std(ddof=1)
    pooled = np.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2)) if (n1 + n2 - 2) > 0 else np.nan
    return (a.mean() - b.mean()) / pooled if pooled and not np.isnan(pooled) else np.nan

In [ ]:

def numeric_mcar_tests(df, alpha=0.05, max_predictors=20, min_group_n=4, fdr_correct=True):
    df = df.copy()
    n = df.shape[0]
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    results = []
    details = {}

    for target in numeric_cols:
        miss_mask = df[target].isnull()
        n_miss = int(miss_mask.sum())
        if n_miss == 0:
            continue
        pct_miss = n_miss / n

        pvals_pearson = {}
        pearson_r = {}
        pvals_t = {}
        t_stats = {}
        cohens = {}
        pvals_mw = {}

        for col in numeric_cols:
            if col == target:
                continue
            gx = df.loc[~miss_mask, col].dropna()
            hx = df.loc[ miss_mask, col].dropna()

            if len(gx) < min_group_n or len(hx) < min_group_n:
                continue

            # Welch t-test
            try:
                tstat, p_t = stats.ttest_ind(gx, hx, equal_var=False, nan_policy='omit')
            except Exception:
                tstat, p_t = np.nan, np.nan

            # Mann-Whitney U (nonparametric)
            try:
                u_stat, p_mw = stats.mannwhitneyu(gx, hx, alternative='two-sided')
            except Exception:
                p_mw = np.nan

            # point-biserial ~ Pearson correlation between numeric col and missing indicator 
            valid_idx = df[col].notnull()
            if valid_idx.sum() >= min_group_n:
                try:
                    r, p_r = stats.pearsonr(df.loc[valid_idx, col], df.loc[valid_idx, target].isnull().astype(int))
                except Exception:
                    r, p_r = np.nan, np.nan
            else:
                r, p_r = np.nan, np.nan

            d = cohens_d(gx, hx)

            pvals_pearson[col] = p_r
            pearson_r[col] = r
            pvals_t[col] = p_t
            t_stats[col] = tstat
            cohens[col] = d
            pvals_mw[col] = p_mw

        pvals_arr = np.array([v for v in pvals_pearson.values() if not pd.isna(v)])
        cols_arr = [k for k, v in pvals_pearson.items() if not pd.isna(v)]
        reject = np.array([])
        pvals_adj = {}
        if fdr_correct and len(pvals_arr) > 0:
            rej, p_adj_arr, _, _ = multipletests(pvals_arr, alpha=alpha, method='fdr_bh')
            reject = rej
            pvals_adj = dict(zip(cols_arr, p_adj_arr))
        else:
            pvals_adj = {k: p for k, p in pvals_pearson.items()}

        sig_after = [col for col in cols_arr if (pvals_adj.get(col, 1.0) < alpha)]
        sig_count = len(sig_after)

        logit_info = {}
        try:
            sort_r = sorted(pearson_r.items(), key=lambda x: 0 if pd.isna(x[1]) else abs(x[1]), reverse=True)
            top_feats = [c for c, _ in sort_r if not pd.isna(pearson_r.get(c))][:max_predictors]
            if len(top_feats) > 0:
                X = df[top_feats].copy().fillna(df[top_feats].mean())
                X = sm.add_constant(X, has_constant='add')
                y = miss_mask.astype(int)
                if y.nunique() == 2 and X.shape[1] >= 1:
                    model = sm.Logit(y, X).fit(disp=0, maxiter=200)
                    pvals_logit = model.pvalues.drop('const', errors='ignore').to_dict()
                    sig_logit = {k: v for k, v in pvals_logit.items() if v < alpha}
                    logit_info = {
                        "n_features_tested": X.shape[1] - 1,
                        "n_significant": len(sig_logit),
                        "significant_predictors": sig_logit
                    }
        except PerfectSeparationError as e:
            logit_info = {"error": f"Perfect separation: {e}"}
        except Exception as e:
            logit_info = {"error": str(e)}

        details[target] = {
            "pearson_pvals": pvals_pearson,
            "pearson_r": pearson_r,
            "pearson_pvals_adj": pvals_adj,
            "t_pvals": pvals_t,
            "t_stats": t_stats,
            "cohens_d": cohens,
            "mannwhitney_pvals": pvals_mw,
            "logit": logit_info,
            "sig_after_fdr": sig_after
        }

        verdict = "evidence_against_MCAR" if sig_count > 0 else "no_strong_evidence_against_MCAR"
        reasons = f"{sig_count} predictors significant after FDR" if sig_count>0 else "no predictors significant after FDR"

        results.append({
            "target": target,
            "n_missing": n_miss,
            "pct_missing": pct_miss,
            "n_predictors_tested": len(pvals_pearson),
            "n_significant_after_fdr": sig_count,
            "verdict": verdict,
            "reasons": reasons
        })

    summary_df = pd.DataFrame(results).sort_values(by="pct_missing", ascending=False).reset_index(drop=True)
    return summary_df, details

In [ ]:
summary, details = numeric_mcar_tests(sample_df)
summary.head()

In [ ]:
count = (summary['verdict'] == 'evidence_against_MCAR').sum()
print(count)

In [ ]:
summary["verdict"].count()


# THE DATA IS MISSING AT RANDOM, IT DEPENDS ON OTHER VARIABLES


In [ ]:
miss_indicators=pd.DataFrame({
    f"{col} missing":sample_df[col].isnull().astype(int)
    for col in sample_df.columns if sample_df[col].isnull().any()
})

In [ ]:
miss_indicators.head()

In [ ]:
from scipy.stats import pearsonr
results=[]
for i in miss_indicators.columns:
    miss_col=miss_indicators[i]
    for col in sample_df.select_dtypes(include=['int64','float64']).columns:
        if col.replace("_missing", "") in i: 
            continue
        valid_idx=sample_df[col].notnull()
        if valid_idx.sum()>5:
            r, p = pearsonr(sample_df.loc[valid_idx, col], miss_col[valid_idx])
            results.append((i, col, r, p))

In [ ]:
results_df = pd.DataFrame(results, columns=['Missing_in', 'Feature', 'Correlation', 'p_value'])
results_df = results_df.sort_values(['Missing_in','p_value'])

# CORRELATION TELLS THE CORRELATION AND P VALUES TELLS HOW STRONG IS THE CORRELATIO (IF P<0.05)

In [ ]:
results_df.head()

In [ ]:
numeric_cols=sample_df.select_dtypes(include=[np.number]).columns.tolist()
miss_indictors=sample_df[numeric_cols].isna().astype(int).add_suffix("missing")

In [ ]:
num_columns = sample_df.shape[1]
print(num_columns)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from xgboost import XGBRegressor

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
# def get_top_predictors(df_combined, miss_col, candidate_cols, top_k=5, min_valid=5):
#     scores = {}
#     target_col = miss_col
#     if target_col not in df_combined.columns:
#         df_combined[target_col] = df_combined[target_col.replace('_missing','')].isnull().astype(int)
#     for col in candidate_cols:
#         if col == target_col.replace('_missing',''):
#             continue
#         valid = df_combined[col].notnull()
#         if valid.sum() < min_valid:
#             continue
#         try:
#             r = df_combined.loc[valid, col].corr(df_combined.loc[valid, target_col])
#         except Exception:
#             r = np.nan
#         if pd.isna(r):
#             continue
#         scores[col] = abs(r)
#     top = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
#     return [c for c,_ in top]

In [ ]:
# def create_lag_features(df, predictors, n_lags=3):
#     lagged = pd.DataFrame(index=df.index)
#     for p in predictors:
#         for lag in range(1, n_lags+1):
#             lagged[f"{p}_lag{lag}"] = df[p].shift(lag)
#     return lagged

In [ ]:
# def causal_impute_column(df, target, predictors, n_lags=3, min_train=30, model=None):
#     df_new = df.copy()
#     if model is None:
#         model = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=-1)
#     preds = [p for p in predictors if p in df_new.columns and p != target]
#     if len(preds) == 0:
#         return df_new, False

#     lagged = create_lag_features(df_new, preds, n_lags)
#     complete_lag_mask = lagged.notnull().all(axis=1)
#     train_mask = complete_lag_mask & df_new[target].notnull()
#     if train_mask.sum() < min_train:
#         return df_new, False

#     X_train = lagged.loc[train_mask]
#     y_train = df_new.loc[train_mask, target].astype(float)

#     model.fit(X_train, y_train)
#     impute_mask = complete_lag_mask & df_new[target].isnull()
#     if impute_mask.sum() == 0:
#         return df_new, False

#     X_pred = lagged.loc[impute_mask]
#     preds_values = model.predict(X_pred)

#     df_new.loc[impute_mask, target] = preds_values
#     changed = df_new[target].notnull().sum() > df[target].notnull().sum()
#     return df_new, changed

In [ ]:
# def iterative_causal_impute(df, n_lags=3, top_k=5, min_train=30, max_passes=5, min_valid=5):
#     df_imp = df.copy()
#     numeric_cols = df_imp.select_dtypes(include=[np.number]).columns.tolist()
#     miss_inds = pd.DataFrame({f"{c}_missing": df_imp[c].isnull().astype(int) for c in numeric_cols}, index=df_imp.index)

#     pass_no = 0
#     overall_changed = True
#     while pass_no < max_passes and overall_changed:
#         pass_no += 1
#         overall_changed = False
#         cols_with_missing = [c for c in numeric_cols if df_imp[c].isnull().any()]
#         if not cols_with_missing:
#             break

#         for target in cols_with_missing:
#             miss_col = f"{target}_missing"
#             combined = pd.concat([df_imp, miss_inds], axis=1)
#             top_preds = get_top_predictors(combined, miss_col, numeric_cols, top_k=top_k, min_valid=min_valid)
#             if not top_preds:
#                 continue
#             df_before_missing = df_imp[target].isnull().sum()
#             df_imp, changed = causal_impute_column(df_imp, target, top_preds, n_lags=n_lags, min_train=min_train)
#             if changed:
#                 overall_changed = True
#                 miss_inds[f"{target}_missing"] = df_imp[target].isnull().astype(int)

#     remaining = df_imp.isnull().sum()
#     fallback_cols = remaining[remaining > 0].index.tolist()
#     for c in fallback_cols:
#         df_imp[c] = df_imp[c].fillna(method='ffill').fillna(method='bfill')  

#     final_missing = df_imp.isnull().sum().sum()
#     return df_imp, final_missing

In [ ]:
# df_imputed, missing_left = iterative_causal_impute(sample_df,
#                                                     n_lags=3, top_k=5, min_train=30,
#                                                     max_passes=6, min_valid=6)

In [ ]:
import os
import polars as pl
import pandas as pd
import numpy as np
import kaggle_evaluation.default_inference_server
import pickle

In [ ]:
FINAL_MODEL=None
FEATURE_COLUMNS=[]
HISTORY_BUFFER=None

In [ ]:
def convert_to_signal(predictions:np.ndarray,multiplier:float=400.0)->np.ndarray:
    signals=predictions*multiplier+1
    return np.clip(signals,0.0,2.0)

In [ ]:
def mice_impute_fast(df, max_iter=3):
    df_imp = df.copy()
    
    numeric_cols = df_imp.select_dtypes(include=[np.number]).columns.tolist()
    
    if not numeric_cols or df_imp[numeric_cols].isnull().sum().sum() == 0:
        return df_imp
    
    lgbm_estimator = LGBMRegressor(
        n_estimators=50,   
        learning_rate=0.1,
        max_depth=4,          
        num_leaves=15,
        random_state=42,
        n_jobs=-1,
        verbose=-1,
        force_col_wise=True   
    )
    
    mice_imputer = IterativeImputer(
        estimator=lgbm_estimator,
        max_iter=max_iter,
        random_state=42,
        initial_strategy='mean',
        verbose=0
    )
    
    try:
        imputed_values = mice_imputer.fit_transform(df_imp[numeric_cols])
        df_imp[numeric_cols] = imputed_values
    except Exception as e:
        print(f"MICE imputation failed: {e}, using simple imputation")
        for col in numeric_cols:
            df_imp[col] = df_imp[col].fillna(df_imp[col].mean()).fillna(0)
    
    remaining = df_imp.isnull().sum()
    if remaining.sum() > 0:
        for col in remaining[remaining > 0].index:
            df_imp[col] = df_imp[col].fillna(method='ffill').fillna(method='bfill').fillna(0)
    return df_imp

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

def train_model():
    global FINAL_MODEL, FEATURE_COLUMNS, HISTORY_BUFFER
    print("Training model...")
    start_time = pd.Timestamp.now()    
    path = "/kaggle/input/hull-tactical-market-prediction"
    print(f"Loading data from {path}")
    training_df = pd.read_csv(f"{path}/train.csv")
    print(f"Loaded {len(training_df)} rows")
    print(f"Missing values before imputation: {training_df.isnull().sum().sum()}")
    
    print("Applying MICE imputation with LightGBM...")
    training_df = mice_impute_fast(training_df, max_iter=3)
    print(f"Imputation done. Remaining NaN: {training_df.isnull().sum().sum()}")
    
    if isinstance(training_df, pd.DataFrame):
        training_df = pl.from_pandas(training_df)
    
    training_df = training_df.rename({'market_forward_excess_returns': 'target'})
    
    feature_cols = [col for col in training_df.columns if col != 'date_id']
    training_df = training_df.with_columns(pl.col(feature_cols).cast(pl.Float64, strict=False))
    
    training_df = training_df.drop_nulls()
    print(f"After dropping nulls: {len(training_df)} rows")
    
    FEATURE_COLUMNS = [col for col in training_df.columns 
                       if col not in ['date_id', 'target', 'forward_returns', 'risk_free_rate']]
    
    print(f"Training with {len(FEATURE_COLUMNS)} features")
    
    X = training_df.select(FEATURE_COLUMNS).to_numpy()
    y = training_df.select('target').to_numpy().ravel()
    
    n_splits = 5  
    tscv = TimeSeriesSplit(n_splits=n_splits)
    cv_scores = []
    cv_rmse = []
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X), 1):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        print(f"\nFold {fold}/{n_splits}")
        print(f"  Train size: {len(train_idx):,} | Val size: {len(val_idx):,}")
        
        model = XGBRegressor(
            n_estimators=200,       
            learning_rate=0.05,   
            max_depth=6,
            num_leaves=31,
            min_child_samples=20,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1,
            verbose=-1,
            force_col_wise=True
        )
        
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_val)
        r2 = r2_score(y_val, y_pred)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        
        cv_scores.append(r2)
        cv_rmse.append(rmse)
        
        print(f"  R² score: {r2:.4f} | RMSE: {rmse:.6f}")
 
    FINAL_MODEL = XGBRegressor(
        n_estimators=200,       
        learning_rate=0.05,   
        max_depth=6,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbose=-1,
        force_col_wise=True
    )
    
    FINAL_MODEL.fit(X, y)
    
    y_pred = FINAL_MODEL.predict(X)
    r2 = r2_score(y, y_pred)
    print(f"Final model trained. R² score: {r2:.4f}")
    
    HISTORY_BUFFER = training_df.select(
        ['date_id'] + FEATURE_COLUMNS + ['target', 'forward_returns', 'risk_free_rate']
    ).tail(50)
    
    elapsed = (pd.Timestamp.now() - start_time).total_seconds()
    print(f"Model training complete in {elapsed:.1f} seconds. Ready for predictions.")
    
    return {
        'cv_r2_mean': np.mean(cv_scores),
        'cv_r2_std': np.std(cv_scores),
        'cv_rmse_mean': np.mean(cv_rmse),
        'cv_scores': cv_scores
    }

In [ ]:
def predict(test: pl.DataFrame) -> float:
    """
    Make a prediction for a single test row.
    
    Args:
        test: A polars DataFrame with one row containing the test features
        
    Returns:
        A float signal value between 0 and 2
    """
    global HISTORY_BUFFER, FINAL_MODEL, FEATURE_COLUMNS
    
    try:
        for col in test.columns:
            if col != 'date_id':
                try:
                    test = test.with_columns(pl.col(col).cast(pl.Float64))
                except:
                    test = test.with_columns(pl.col(col).cast(pl.Utf8).cast(pl.Float64))
        
        rename_mapping = {}
        if 'lagged_forward_returns' in test.columns:
            rename_mapping['lagged_forward_returns'] = 'forward_returns'
        if 'lagged_risk_free_rate' in test.columns:
            rename_mapping['lagged_risk_free_rate'] = 'risk_free_rate'
        if 'lagged_market_forward_excess_returns' in test.columns:
            rename_mapping['lagged_market_forward_excess_returns'] = 'target'
        
        if rename_mapping:
            test = test.rename(rename_mapping)        
        if 'is_scored' in test.columns:
            test = test.drop('is_scored')
        
        for col in HISTORY_BUFFER.columns:
            if col not in test.columns:
                col_dtype = HISTORY_BUFFER[col].dtype
                test = test.with_columns(pl.lit(0.0).cast(col_dtype).alias(col))
        
        for col in test.columns:
            if col in HISTORY_BUFFER.columns:
                if test[col].dtype != HISTORY_BUFFER[col].dtype:
                    test = test.with_columns(pl.col(col).cast(HISTORY_BUFFER[col].dtype))
        
        test = test.select(HISTORY_BUFFER.columns)        
        HISTORY_BUFFER = pl.concat([HISTORY_BUFFER, test], how="vertical")        
        if len(HISTORY_BUFFER) > 100:
            HISTORY_BUFFER = HISTORY_BUFFER.tail(100)        
        latest_row = HISTORY_BUFFER.tail(1)
        X_test = latest_row.select(FEATURE_COLUMNS).to_numpy()        
        X_test = np.nan_to_num(X_test, nan=0.0)        
        raw_prediction = FINAL_MODEL.predict(X_test)[0]        
        signal = convert_to_signal(np.array([raw_prediction]))[0]
        return float(signal)
    except Exception as e:
        print(f"Error in prediction: {e}")
        return 1.0

In [ ]:
train_model()

In [ ]:
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("Running in competition mode")
    inference_server.serve()
else:
    print("Running in local gateway mode")
    inference_server.run_local_gateway(
        ('/kaggle/input/hull-tactical-market-prediction/',)
    )